Double Play Analytics

In [1]:
#Import Packages
import pandas as pd

In [2]:
file_paths = ["data/gl2010.txt", "data/gl2011.txt", "data/gl2012.txt", "data/gl2013.txt", "data/gl2014.txt",
              "data/gl2015.txt", "data/gl2016.txt", "data/gl2017.txt", "data/gl2018.txt", "data/gl2019.txt",
              "data/gl2020.txt", "data/gl2021.txt", "data/gl2022.txt", "data/gl2023.txt"]

In [3]:
dfs=[]

In [4]:
for file_path in file_paths:
    with open(file_path, "r") as f:
        # Read lines from the file
        data = f.readlines()
        # Split each line by comma and create a list of lists
        data_split = [line.strip().split(",") for line in data]
        # Create DataFrame from the data
        df = pd.DataFrame(data_split)
        # Append DataFrame to the list
        dfs.append(df)

In [5]:
# Concatenate all DataFrames in the list
result_df = pd.concat(dfs, ignore_index=True)

In [6]:
result_df.head(20)

,0,1,2,3,4,5,6,7,8,9,...,169,170,171,172,173,174,175,176,177,178
0,"""20100404""","""0""","""Sun""","""NYA""","""AL""",1,"""BOS""","""AL""",1,7,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,"""20100405""","""0""","""Mon""","""MIN""","""AL""",1,"""ANA""","""AL""",1,3,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,"""20100405""","""0""","""Mon""","""CLE""","""AL""",1,"""CHA""","""AL""",1,0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,"""20100405""","""0""","""Mon""","""DET""","""AL""",1,"""KCA""","""AL""",1,8,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,"""20100405""","""0""","""Mon""","""SEA""","""AL""",1,"""OAK""","""AL""",1,5,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
5,"""20100405""","""0""","""Mon""","""TOR""","""AL""",1,"""TEX""","""AL""",1,4,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
6,"""20100405""","""0""","""Mon""","""SDN""","""NL""",1,"""ARI""","""NL""",1,3,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
7,"""20100405""","""0""","""Mon""","""CHN""","""NL""",1,"""ATL""","""NL""",1,5,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
8,"""20100405""","""0""","""Mon""","""SLN""","""NL""",1,"""CIN""","""NL""",1,11,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
9,"""20100405""","""0""","""Mon""","""SFN""","""NL""",1,"""HOU""","""NL""",1,5,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
result_df.shape

(32484, 179)

In [8]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32484 entries, 0 to 32483
Columns: 179 entries, 0 to 178
dtypes: object(179)
memory usage: 44.4+ MB


In [9]:
new_column_names = {
    0: 'date',
    1: 'num_games',
    2: 'day_of_week',
    3: 'team_visiting',
    4: 'league_visiting',
    5: 'game_num_visiting',
    6: 'team_home',
    7: 'league_home',
    8: 'game_num_home',
    9: 'score_visiting',
    10: 'score_home',
    11: 'outs_in_game',
    12: 'time_of_day',
    13: 'game_completed',
    14: 'forfeit',
    15: 'protest',
    16: 'park_id',
    17: 'attendance',
    18: 'length_min',
    19: 'line_score_visiting',
    20: 'line_score_home',    
    21: 'at_bats_visiting',
    22: 'hits_visiting',
    23: 'double_visiting',
    24: 'triple_visiting',
    25: 'home_run_visiting',
    26: 'rbi_visiting',
    27: 'sacrifice_hit_visiting',
    28: 'sacrifine_fly_visiting',
    29: 'hit_by_pitch_visiting',
    30: 'walk_visiting',
    31: 'intent_walk_visiting',
    32: 'strikeout_visiting',
    33: 'stolen_base_visiting',
    34: 'caught_stealing_visiting',
    35: 'grounded_into_double_plays_visiting',
    36: 'first_catcher_interfere_visiting',
    37: 'left_on_base_visiting',
    38: 'pitchers_used_visiting',
    39: 'individual_earned_runs_visiting',
    40: 'team_earned_runs_visiting',
    41: 'wild_pitches_visiting',
    42: 'balks_visiting',
    43: 'putouts_visiting',   
    44: 'assists_visiting',
    45: 'errors_visiting',
    46: 'passed_balls_visiting',
    47: 'double_def_visiting',
    48: 'triple_def_visiting',       
    49: 'at_bats_home',
    50: 'hits_home',
    51: 'double_home',
    52: 'triple_home',
    53: 'home_run_home',
    54: 'rbi_home',
    55: 'sacrifice_hit_home',
    56: 'sacrifine_fly_home',
    57: 'hit_by_pitch_home',
    58: 'walk_home',
    59: 'intent_walk_home',
    60: 'strikeout_home',
    61: 'stolen_base_home',
    62: 'caught_stealing_home',
    63: 'grounded_into_double_plays_home',
    64: 'first_catcher_interfere_home',
    65: 'left_on_base_home',
    66: 'pitchers_used_home',
    67: 'individual_earned_runs_home',
    68: 'team_earned_runs_home',
    69: 'wild_pitches_home',
    70: 'balks_home',
    71: 'putouts_home',
    72: 'assists_home',
    73: 'errors_home',
    74: 'passed_balls_home',
    75: 'double_def_home',
    76: 'triple_def_home',    
    77: 'home_ump_id',
    78: 'home_ump_name',
    79: '1B_ump_id',
    80: '1B_ump_name',
    81: '2B_ump_id',
    82: '2B_ump_name',
    83: '3B_ump_id',
    84: '3B_ump_name',
    85: 'LF_ump_id',
    86: 'LF_ump_name',
    87: 'RF_ump_id',
    88: 'RF_ump_name',
    89: 'manager_id_visiting',
    90: 'manager_name_visiting',
    91: 'manager_id_home',
    92: 'manager_name_home',
    93: 'winning_pitcher_id',
    94: 'winning_pitcher_name',
    95: 'losing_pitcher_id',
    96: 'losing_pitcher_name',
    97: 'saving_pitcher_id',
    98: 'saving_pitcher_name',
    99: 'game_winning_rbi_id',
    100: 'game_winning_rbi_name',
    101: 'starting_pitcher_id_visiting',
    102: 'starting_pitcher_name_visiting',
    103: 'starting_pitcher_id_home',
    104: 'starting_pitcher_name_home',
    105: 'player1_id_visiting',
    106: 'player1_name_visiting',
    107: 'player1_position_visiting',
    108: 'player2_id_visiting',
    109: 'player2_name_visiting',
    110: 'player2_position_visiting',
    111: 'player3_id_visiting',
    112: 'player3_name_visiting',
    113: 'player3_position_visiting',
    114: 'player4_id_visiting',
    115: 'player4_name_visiting',
    116: 'player4_position_visiting',
    117: 'player5_id_visiting',
    118: 'player5_name_visiting',
    119: 'player5_position_visiting',
    120: 'player6_id_visiting',
    121: 'player6_name_visiting',
    122: 'player6_position_visiting',
    123: 'player7_id_visiting',
    124: 'player7_name_visiting',
    125: 'player7_position_visiting',
    126: 'player8_id_visiting',
    127: 'player8_name_visiting',
    128: 'player8_position_visiting',
    129: 'player9_id_visiting',
    130: 'player9_name_visiting',
    131: 'player9_position_visiting',    
    132: 'player1_id_home',
    133: 'player1_name_home',
    134: 'player1_position_home',
    135: 'player2_id_home',
    136: 'player2_name_home',
    137: 'player2_position_home',
    138: 'player3_id_home',
    139: 'player3_name_home',
    140: 'player3_position_home',
    141: 'player4_id_home',
    142: 'player4_name_home',
    143: 'player4_position_home',
    144: 'player5_id_home',
    145: 'player5_name_home',
    146: 'player5_position_home',
    147: 'player6_id_home',
    148: 'player6_name_home',
    149: 'player6_position_home',
    150: 'player7_id_home',
    151: 'player7_name_home',
    152: 'player7_position_home',
    153: 'player8_id_home',
    154: 'player8_name_home',
    155: 'player8_position_home',
    156: 'player9_id_home',
    157: 'player9_name_home',
    158: 'player9_position_home'
}


# Rename columns
result_df.rename(columns=new_column_names, inplace=True)

Other feature engineering
    - run differential 

In [10]:
result_df.head()

,date,num_games,day_of_week,team_visiting,league_visiting,game_num_visiting,team_home,league_home,game_num_home,score_visiting,...,169,170,171,172,173,174,175,176,177,178
0,"""20100404""","""0""","""Sun""","""NYA""","""AL""",1,"""BOS""","""AL""",1,7,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
1,"""20100405""","""0""","""Mon""","""MIN""","""AL""",1,"""ANA""","""AL""",1,3,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,"""20100405""","""0""","""Mon""","""CLE""","""AL""",1,"""CHA""","""AL""",1,0,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,"""20100405""","""0""","""Mon""","""DET""","""AL""",1,"""KCA""","""AL""",1,8,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,"""20100405""","""0""","""Mon""","""SEA""","""AL""",1,"""OAK""","""AL""",1,5,...,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
result_df["day_of_week"].value_counts()

"Sat"    5352
"Sun"    5259
"Fri"    5128
"Tue"    4935
"Wed"    4931
"Mon"    3459
"Thu"    3420
Name: day_of_week, dtype: int64

In [13]:
result_df.describe()


,date,num_games,day_of_week,team_visiting,league_visiting,game_num_visiting,team_home,league_home,game_num_home,score_visiting,...,169,170,171,172,173,174,175,176,177,178
count,32484,32484,32484,32484,32484,32484,32484,32484,32484,32484,...,60,17,14,14,10,9,9,9,1,1
unique,2417,3,7,31,2,163,31,2,163,27,...,14,9,6,10,7,2,9,2,1,1
top,"""20200904""","""0""","""Sat""","""TOR""","""NL""",36,"""PHI""","""NL""",19,3,...,"""Y""",ump2b,ump2b,"""Y""",6,ump3b,"willr901""","""Y""","(None)""","""Y"""
freq,20,31518,5352,1091,16479,222,1088,16491,225,4625,...,43,3,9,4,3,8,1,8,1,1


In [14]:
def compare_and_get_value(row):
    if row['score_visiting'] > row['score_home']:
        return row['team_visiting']
    else:
        return row['team_home']

# Create a new feature based on the comparison
result_df['winning_team'] = result_df.apply(compare_and_get_value, axis=1)

#################check to see if there was ever a tie or null value in this category

In [15]:
result_df['winning_team'].value_counts()

"LAN"    1226
"NYA"    1162
"ATL"    1155
"TBA"    1151
"MIL"    1144
"SLN"    1136
"CLE"    1117
"SFN"    1101
"BOS"    1100
"PHI"    1093
"TOR"    1088
"OAK"    1083
"ANA"    1079
"CHA"    1078
"CHN"    1078
"HOU"    1077
"WAS"    1076
"NYN"    1073
"DET"    1071
"SEA"    1071
"CIN"    1066
"TEX"    1059
"BAL"    1056
"MIN"    1056
"PIT"    1043
"SDN"    1033
"COL"    1018
"KCA"    1016
"ARI"     995
"MIA"     825
"FLO"     158
Name: winning_team, dtype: int64

In [16]:
#################I want to see .info for all features, want to look at missing values-which features have missing. Below code not working yet

# Setting display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Then, display the info of the DataFrame
result_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32484 entries, 0 to 32483
Columns: 180 entries, date to winning_team
dtypes: object(180)
memory usage: 44.6+ MB


In [ ]:
#with open("data/gl2010.txt", "r") as f:
#    data1=f.readlines()

In [ ]:
#with open("data/gl2011.txt", "r") as f:
#    data2=f.readlines()

In [ ]:
#data1 = [line.strip().split(',') for line in data1]
#data2 = [line.strip().split(',') for line in data2]

In [ ]:
#df1 = pd.DataFrame(data1)
#df2 = pd.DataFrame(data2)

In [ ]:
#merged_df = pd.concat([df1, df2], axis=0)

In [ ]:
#merged_df.head()

In [ ]:
#merged_df.shape

In [ ]:
#Import Data
#df=pd.read_csv(data/)

Data Notes: 
- COVID years
    - 2020 less games